In [23]:
from random import randint, uniform, choice
from util import generate_problem, verify_witness

INF = 1e9

## Firstly we need a solver for the STP problem.

note here we are given a distance graph
(the generation is done in the genetic part, so we can reuse memory and save time when an interval selection is changed.)

In [2]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(E[i][j], E[i][k] + E[k][j])

    return E

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def solve_stp(graph):
    ''' graph is the constraint adjacency matrix '''
    
    d_graph = generate_d_graph(graph)
    
    return consistent(d_graph), get_min_sol(d_graph)

## The genetic part:

In [52]:
def random_gene(T):
    interval_selection = [randint(0, len(constr['intervals'])-1) for constr in T]
    return interval_selection

def update_graph(graph, gene):
    # set up graph using index 0 everywhere
    # so later we can change interval in O(1) time
    # (saves memory and time, and this same approach is especially beneficial in meta_walk)
    for i, constr in enumerate(T):
        interval_index = gene[i]
        
        i, j = constr['i'], constr['j']
        interval = constr['intervals'][interval_index]
        
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]
    
    return graph

def update_graph_at_constraint(graph, gene, constraint_index):
    ''' update_graph but updates only one constraint '''
    interval_index = gene[i]
    
    constr = T[constraint_index]
    
    i, j = constr['i'], constr['j']
    interval = constr['intervals'][interval_index]

    graph[i][j] = interval[1]
    graph[j][i] = -interval[0]
        
    return graph

def walk_gene(gene, T):
    # ignore all that have no choice
    # create list of plausible indices and filter it.
    candidate_indices = [j for j in range(len(T))]
    candidate_indices = list(filter(
        lambda x: len(T[x]['intervals']) > 1,
        candidate_indices
    ))
    
    idx = choice(candidate_indices)
    gene[idx] = randint(0, len(T[idx]['intervals'])-1)
    
    return gene

def meta_walk(T, max_iterations, max_flips):
    num_variables = max([max(t['i'], t['j']) for t in T])
    graph = discrete_graph(num_variables+1)
    
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T)
        graph = update_graph(graph, gene)
        
        for j in range(max_flips):
            # at each iteration we modify

            is_consistent, witness = solve_stp(graph)
            gene_failed = verify_witness(witness, T)

            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed

            if is_consistent:
                break
                
            gene = walk_gene(gene, T)
        
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [53]:
T = generate_problem()
meta_walk(T, 50, 50)

num iterations: 50
best gene: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 3, 1, 0, 0, 3]
constraints failed: 0 out of: 21
failed constraints: []
